In [98]:
import pandas as pd

In [99]:
# df = pd.read_csv(r'C:\Users\ventu\WSL\Python\Data_Science_Projects\Used Phones\storage.csv')
# df.head()

In [100]:
df = pd.read_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv')
df = df.set_index('Listing')

In [101]:
df.insert(12, 'Title', '')

In [103]:
df = df

In [104]:
df.to_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv')

In [105]:
df2 = pd.read_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv')
df.head()

,Model,Platform,Carrier,Color,Storage,ListDate,ExpiredDate,SaleDate,Views,Quantity,Price,Condition,Title,Description,Damage,Sold
Listing,,,,,,,,,,,,,,,,
LUJS17403,Apple iPhone 11,iOS,AT&T,Black,64 GB,"Sep 08, 2020",NaN,"Sep 15, 2020",24,1,645,Mint,,\nTested 100% working. Comes with full accesso...,NaN,Yes
LUDI07758,Apple iPhone 11,iOS,AT&T,Purple,64 GB,"Mar 27, 2020",NaN,"Sep 14, 2020",335,0,658,Mint,,\nThis phone is in MINT condition. It is full...,NaN,Yes
LUJX60358,Apple iPhone 11,iOS,AT&T,Purple,128 GB,"Sep 14, 2020",NaN,"Sep 14, 2020",15,1,590,Good,,"\nAT&T Apple iPhone 11 128GB UNLOCKED , GREAT ...","\nlight signs of wear, barely seen.\n",Yes
LUIX98388,Apple iPhone 11 Pro Max,iOS,AT&T,Gray,256 GB,"Aug 19, 2020",NaN,NaN,189,1,1079,Mint,,\nLike New Device» Mint Condition» Generic Box...,NaN,NaN
LUJY83673,Apple iPhone 11 Pro Max,iOS,AT&T,Green,64 GB,"Sep 14, 2020","Sep 24, 2020",NaN,21,1,855,Mint,,\niPhone 11 Pro Max - green color. Like new co...,NaN,NaN
